<a href="https://colab.research.google.com/github/NicolasZarateC/Analitica-de-Negocios/blob/main/Reto_2_TechSales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

#Carga desde drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nxl ='/content/drive/MyDrive/2. TechSalesReport_Int_L.xlsx'

1. **Descripcion Variables**
**Business**: Esta cateogora menciona dos tipos de producto, hardware (0)o software (1)
**Age**: Edad del empleado
**Female**: Si el genero del empleado es mujer (1) o otro caso (0)
**Years**: La cantidad de años que lleva un empleador en la empresa
**College**: Si el empleado tiene un titulo (1) universitario o no (0)
**Certificates**: La cantidad de certificados profesionales relevantes a la ocupación que tiene cada persona de la empresa
**Feedback**: De 0 a 4 (0 siendo el más bajo y el 4 el más alto) es una puntuación de una evaluación anual otorgado por los compañeros y supervisores de los empleados
**Salary**: El sueldo de base de cada empleado en terminos anuales
**NPS**: Un indicador de la satisfacción del cliente (NPS = Net promoter score , o en español puntaje neto de promotor)

In [ ]:
XDB=pd.read_excel(nxl,sheet_name=0)
XDB.head(100)

#No tomamos en cuenta estas variables
XDB.drop(['Personality'],axis=1,inplace=True)
XDB.drop(['Sales_Rep'],axis=1,inplace=True)

#Cambiar a variables numericas
XDB['Business']=pd.factorize(XDB['Business'])[0]
XDB['College'] = XDB['College'].map({'Yes': 1, 'No': 0})

#Selecion variables
XD=XDB[['Business', 'Age', 'Female', 'Years', 'Certificates', 'Feedback', 'Salary', 'NPS']]
yd=XDB[['College']]

2. Niveles de analisis con arbol de decision

In [ ]:
#Arbol de decision
mar=DecisionTreeClassifier(criterion='gini',max_depth=4)
mar.fit(XD,yd)

from sklearn.tree import export_graphviz
from pydotplus import graph_from_dot_data

vs=['Business', 'Age', 'Female', 'Years', 'Certificates', 'Feedback', 'Salary', 'NPS']

dot_data=export_graphviz(mar,feature_names=vs)
graph=graph_from_dot_data(dot_data)
graph.write_png('1. ArbolTechSales.png')

True

3. Caracterizacion de datos (datos totales y numero de personas inscritas en la universidad y numero de personas no inscritas en la universidad)

In [ ]:
#Caraterizacion datos
from os import ctermid
from sklearn.naive_bayes import GaussianNB

#¿Cuál es el número de datos total de la base de datos?
mnb=GaussianNB()
mnb.fit(XD,yd)

ct=mnb.class_count_
print("El total de personas es:",len(XD))

#Separacion datos
print("Como estan separados los datos",ct)

#Numero de personas no inscritas: 4470
#Numero de personas inscritas: 17520

El total de personas es: 21990
Como estan separados los datos [ 4470. 17520.]


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


4. Nodos puros


Mirando el árbol de decisión, viendo que no hay nodos donde el gini es igual a 0, vemos que **NO hay nodos puros en esta base de datos**.

6. Matriz de confusión

In [ ]:
#Matriz Confusion
ydp=mar.predict(XD)
cm=confusion_matrix(yd,ydp)
print(cm)

VN=cm[0,0];FP=cm[0,1];FN=cm[1,0];VP=cm[1,1]

Exactitud=(VP+VN)/(VP+VN+FP+FN)
print('La exactitud es:\n',Exactitud)
Sensibilidad=VP/(VP+FN)
print('La sensibilidad es:\n',Sensibilidad)
Especificidad=VN/(VN+FP)
print('La especificidad es:\n',Especificidad)
Precision=VP/(VP+FP)
print('La precision es:\n',Precision)
TasaError=(FP+FN)/(VP+VN+FP+FN)
print('La tasa de error es:\n',TasaError)
PredNeg=VN/(VN+FN)
print('La tasa de prediccion negativa es:\n',PredNeg)

[[  114  4356]
 [   69 17451]]
La exactitud es:
 0.7987721691678036
La sensibilidad es:
 0.9960616438356165
La especificidad es:
 0.025503355704697986
La precision es:
 0.8002476269087908
La tasa de error es:
 0.20122783083219645
La tasa de prediccion negativa es:
 0.6229508196721312


5. Pronóstico y 7. Informe de pronóstico para excel

In [ ]:
#Pronóstico
XDB2=pd.read_excel(nxl,sheet_name=1)
XDB2['Business']=pd.factorize(XDB2['Business'])[0]
XD2=XDB2[['Business', 'Age', 'Female', 'Years', 'Certificates', 'Feedback', 'Salary', 'NPS']]

#Se hace el pronostico para estas personas
ydp2=mar.predict(XD2)
print('El pronostico es:\n',ydp2)

#Crear el informe con la PreApr
dfp=pd.DataFrame(np.column_stack((XD2,ydp2)))
dfp.columns=['Business', 'Age', 'Female', 'Years', 'Certificates', 'Feedback', 'Salary', 'NPS', 'College',]
dfp.head(10)
dfp.to_excel('PronosticoTechSales.xlsx')

print('El pronostico de los que no tienen titulo universitario son:',len(np.where(ydp2==0)[0]))
print('El pronostico de los que si tienen titulo universitario son:',len(np.where(ydp2==1)[0]))

#Debido a la predominancia de los que si tienen titulo universitario, el pronostico de los que NO da 0
#No hay puros pero usando el de 0.075

El pronostico es:
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
El pronostico de los que no tienen titulo universitario son: 0
El pronostico de los que si tienen titulo universitario son: 16


**Analisis de Resultados**


Para la implementación del modelo de Arbol de Decisión, se tomaron un total de 9 variables, 8 variables de entrada, entre las que se destacan negocio, edad, si la persona es de genero femenino, años de trabajo en la empresa, certificados relevante, puntuación de retroalimentación, salario y puntaje neto promotor, y a la vez una variable de salida que permite evaluar si el empleado tiene o no un titulo universitario. Para la construcción del modelo  se tuvieron un total de 21990 datos divididos entre 4470 empleados sin titulo universitario y 17520. Luego de la ejecución del modelo se logró una exactitud cercana al 79% promedio, lo cual por ser más del 75% indica que la calidad del pronostico es muy buena. Es importante destacar que la precision del modelo es más del 75% (80%) y que el coeficiente de sensibilidad estuvo casi a un 100% (99%) demostrandonos otra vez el buen funcionamiento de este modelo. Igual con la tasa de error del 20%, corraboramos que se puede confiar en este modelo debido a que la predicción de los errores es baja.

Además, aunque contemos con la ausencia de nodos puros, vemos que el nodo base tiene un indice gini de 0.075 que de igual manera es un número muy cercano a 0 y por eso se puede tomar como base. Llegamos a este nodo si seguimos la regla: IF Salary>=64100 and Salary>96500 and Certificates<4.5 and Feedback<3.495 con un total de 1056 empleados donde 41 no tienen titulo universitario (4%) y 1015 si tienen titulo (96%).

Se ve una clara diferencia respecto a la cantidad de empleados que no tienen titulo universitario y los que si. Con un total de 21990 empleados siendo evaluados si tienen titulo universitario o no, vemos que 4470 de estos empleados no cuentan con el titulo (correspondiente a un 20%  de los datos totales) pero 17520 si tienen un titutlo (correspondiente a alrededor de 80% de los datos totales). Esta gran diferencia entre estos dos datos explica el pronostico del modelo, ya que tiene a siempre ser 1 por la brecha tan grande que hay entre los dos datos evaluados.
